In [1]:
from pydantic import BaseModel, Field, ValidationError

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA, LLMChain
from langchain.memory import ConversationBufferMemory

from langchain.agents import initialize_agent
from langchain.agents import AgentType, ZeroShotAgent, AgentExecutor

In [2]:
#class DocumentInput(BaseModel):
 #   question: str = None


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

tools = []
files = [
    # https://abc.xyz/investor/static/pdf/2023Q1_alphabet_earnings_release.pdf
    {
        "name": "Resume Vega",
        "path": "PDFS/Sebastian vega Cv.pdf",
    },
    # https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2023-Update
    {
        "name": "Resume Omar",
        "path": "PDFS/Omar_Macias_Resume.pdf",
    },
    {
        "name": "Resume Diego Mojarro",
        "path": "PDFS/CV_Diego_Mojarro 2.pdf",
    },
    {
        "name": "Resume Jorge Plasencia",
        "path": "PDFS/CV_JorgePlasencia.pdf",
    },
    {
        "name": "Resume Iker",
        "path": "PDFS/Iker CV.pdf",
    },
    {
        "name": "Resume Gael Venegas",
        "path": "PDFS/Gael Venegas Resume.pdf",
    },
]



In [3]:
for file in files:
    loader = PyPDFLoader(file["path"])
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

    # Wrap retrievers in a Tool
    tools.append(
        Tool(
            #args_schema=DocumentInput(),
            name=file["name"],
            description=f"useful when you want to answer questions about {file['name']}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    )   

In [4]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4",
)

agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
)

In [8]:
agent.run("Dime el proyecto mas interesante de Diego y por que ")



> Entering new AgentExecutor chain...
Necesito revisar el currículum de Diego Mojarro para responder a esta pregunta.
Action: Resume Diego Mojarro
Action Input: Proyectos de Diego Mojarro
Observation: {'query': 'Proyectos de Diego Mojarro', 'result': 'Algunos de los proyectos de Diego Mojarro son:\n\n1. Corajillo Supply Chain: Desarrolló una aplicación full-stack en Next.js, Tailwind CSS, AWS RDS, PostgreSql y Node.js. Optimizó la gestión de la cadena de suministro mediante el cálculo de la producción faltante, lo que resultó en un aumento del 25% en la productividad. Este proyecto fue el ganador entre 16 equipos.\n\n2. relikz.xyz: Desarrolló una aplicación web3 que crea y ejecuta información blockchain en el cliente. Diseñó y desarrolló componentes de React para mejorar la experiencia del usuario. Utilizó Next.js, Tailwind CSS, Moralis, Solidity y Node.js.\n\n3. IBM Labor Recovery Expenses Tool: Implementó una aplicación de pila completa para IBM, que permite a los gerentes visualiz

'El proyecto más interesante de Diego podría ser "Corajillo Supply Chain". En este proyecto, Diego desarrolló una aplicación full-stack y optimizó la gestión de la cadena de suministro mediante el cálculo de la producción faltante, lo que resultó en un aumento del 25% en la productividad. Este proyecto fue el ganador entre 16 equipos. Sin embargo, la elección del proyecto más interesante puede ser subjetiva y depender de los intereses individuales.'